# Coding Attention Mechanism

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import lightning as L

device = torch.device("cuda")
L.seed_everything(42)

Seed set to 42


42

In [3]:
%reload_ext watermark
%watermark -a "Rodrigo W Pisaia" --iversions --python

Author: Rodrigo W Pisaia

Python implementation: CPython
Python version       : 3.10.16
IPython version      : 8.30.0

pandas    : 2.2.3
torch     : 2.6.0+cu126
numpy     : 2.1.3
lightning : 2.5.0.post0
matplotlib: 3.10.1



* 1) Simplified self-attention
* 2) Self-attention
* 3) Causal attention
* 4) Multi-head Attention

![causal_self_attention](../images/image.png)

In [4]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [5]:
torch.dot(torch.tensor(inputs[1]).detach(),torch.tensor(inputs[1]).detach()) # the self weight doesn't have value 1, because it is basically the square


C:\Users\RWP\AppData\Local\Temp\ipykernel_7504\2096973130.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.dot(torch.tensor(inputs[1]).detach(),torch.tensor(inputs[1]).detach()) # the self weight doesn't have value 1, because it is basically the square


tensor(1.4950)

In [6]:
# calculate attention values with journey
query = inputs[1] # [0.55, 0.87, 0.66]
attn_scores_2 = torch.empty(inputs.shape[0]) # 6
for i, x_in in enumerate(inputs):
    print(i, x_in)
    break

0 tensor([0.4300, 0.1500, 0.8900])


In [7]:
for i, x_in in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_in, query)
attn_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [8]:
# internaly it will be a matrix multiplication
res = 0.
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(0.43*0.55+0.15*0.87+0.89*0.66) # explicit calculation of dot product
print(res)
print(torch.dot(inputs[0], query)) # dot product 

0.9544
tensor(0.9544)
tensor(0.9544)


![Attention Weights](../images/attention_weights.png)

In [ ]:
attn_weights_2_tmp = attn_scores_2/torch.sum(attn_scores_2)
print(attn_weights_2_tmp)
print(torch.sum(attn_weights_2_tmp)) # the sum of normalized attention weights is 1

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)


In [24]:
def softmax_naive(x):
    
    return torch.exp(x) / torch.exp(x).sum(dim=0)
    # return torch.exp(x)/torch.sum(torch.exp(x))

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)
